In [ ]:
from __future__ import annotations

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from st_aggrid import AgGrid, GridOptionsBuilder
import datetime as dt
import textwrap
import html

# To Download Project Template 
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from io import BytesIO
import pandas as pd





from io import BytesIO
from typing import Iterable, Optional

import pandas as pd
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter


def df_to_excel_bytes(df,
                      sheet_name= "Sheet1",
                      default_max_width= 30,
                      long_columns=[],
                      long_max_width= 80):
    """
    
    
    """
    min_width = 10
    padding = 2
    wrap_vertical_align= "top"
    freeze_header= True,

    long_columns_set = set(long_columns or [])

    buffer = BytesIO()
    with pd.ExcelWriter(buffer, engine="openpyxl") as writer:
        safe_sheet = sheet_name[:31]  # Excel sheet name limit
        df.to_excel(writer, index=False, sheet_name=safe_sheet)
        ws = writer.sheets[safe_sheet]

        if freeze_header:
            ws.freeze_panes = "A2"

        # Predefine alignments (reuse objects)
        wrap_align = Alignment(wrap_text=True, vertical=wrap_vertical_align)
        no_wrap_align = Alignment(wrap_text=False, vertical=wrap_vertical_align)

        for i, col in enumerate(df.columns, start=1):
            col_letter = get_column_letter(i)

            # Compute max string length in this column (including header)
            ser = df[col].astype(str).fillna("")
            max_len = max(len(str(col)), int(ser.map(len).max()) if len(ser) else 0)

            # Choose cap
            cap = long_max_width if col in long_columns_set else default_max_width

            # Proposed width (with padding)
            proposed = max_len + padding

            # Final width with min + cap
            final_width = max(min_width, min(proposed, cap))
            ws.column_dimensions[col_letter].width = final_width

            # Wrap if we had to cap (meaning content would exceed the allowed width)
            should_wrap = proposed > cap
            if should_wrap:
                # Apply wrap to entire column, incl header
                for cell in ws[col_letter]:
                    cell.alignment = wrap_align
            else:
                # Optional: set vertical alignment consistently
                for cell in ws[col_letter]:
                    cell.alignment = no_wrap_align

    return buffer.getvalue()


In [24]:
display(notes[notes['Word']=='ML Process'][['Categorization','Definition']])

,Categorization,Definition
0,Problem Definition,Project Goal Explicitly stated in 3-4 Sentence...
1,Problem Definition,Explicilty Defined Business Delieverables
2,Problem Definition,Explicilty Defined Expected Outcomes and KPIs
4,Data Preparation,Identify Data Elements which appear relevant (...
5,Data Preparation,"Label, and define data (Semantic, Functional a..."
6,Data Preparation,"Document data omissions, known data quality is..."
7,Feature Engineering,
8,Feature Selection,"Define how feature evaluation will occur, expl..."
20,Hyperparameter Tuning,
22,"Bias, Fairness and Ethics",


In [18]:
notes = pd.read_csv(google_notes_csv).fillna('')
notes = notes[['Category','Categorization','Word','Definition']].copy()


In [10]:
d_learning_notes[d_learning_notes['Definition'].fillna('').str.contains('Test')]

,Category,Categorization,Word,Definition,COUNT
430,Machine Learning,Monitoring,Clustering,This is a Test: Does this Show???,1.0


In [8]:
definitions[definitions['Word'].str.contains('Test',case=False)]

,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class
308,Monitoring,Clustering,This is a Test,Does this Show???,,,,,,,


In [25]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

google_notes_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv'
google_definition_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv'

notes = pd.read_csv(google_notes_csv).fillna('')
notes = notes[['Category','Categorization','Word','Definition']].copy()

definitions = pd.read_csv(google_definition_csv).fillna('')

ml_def = definitions.copy()
ml_notes = notes[notes['Category'].isin(['Machine Learning','Visualizations'])]

from df_processing import final_dataset_for_markdown
d_learning_notes = final_dataset_for_markdown(notes,definitions)


In [17]:
d_learning_notes[d_learning_notes['Definition'].str.contains('Test')]

,Category,Categorization,Word,Definition,COUNT
336,Machine Learning,Monitoring,Clustering,This is a Test: Does this Show???,1.0


In [2]:
d_learning_notes

,Category,Categorization,Word,Definition,COUNT
126,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...,1.0
131,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...,1.0
132,Machine Learning,Definition,Bias,Bias refers to the systematic error in a model...,1.0
133,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...,1.0
134,Machine Learning,Definition,Cross Validation,Model evaluation technique used to assess how ...,1.0
...,...,...,...,...,...
412,NaN,Machine Learning,Clustering,Centroid-based clustering: Centroid-based clus...,NaN
415,NaN,Machine Learning,Clustering,Density-based clustering: Density-based cluste...,NaN
418,NaN,Machine Learning,Clustering,Probabilistic clustering: Probabilistic cluste...,NaN
420,NaN,Machine Learning,Clustering,Graph-based clustering: Graph-based clustering...,NaN


In [24]:
def search_df_word(df,word,columns=None,case_sensitive=False,print_=True):
    '''
    
    
    '''

    # If User does not define which columns they wish to search, search all.
    if not columns:
        columns = df.columns.tolist()

    final_dict = {}
    final_df = pd.DataFrame()

    for column in columns:
        match_df = df[df[column].fillna('').str.contains(word,case=case_sensitive)]
        if len(match_df)==0:
            final_dict[column] = 'No Matches'
        else:
            final_word = ""
            for word in match_df[column]:
                final_word += word + ","
        
            final_dict[column] = final_word
            final_df = pd.concat([final_df,match_df])

    if print_:
        try:
            print(final_df)
        except:
            pass
            
    return final_df, final_dict

#final_df,final_dict = search_df_word(d_learning_notes,'regularization')
final_df,final_dict = search_df_word(d_learning_notes,'validation')

    

             Category Categorization                   Word  \
307  Machine Learning     Validation             Definition   
308  Machine Learning     Validation                   Goal   
309  Machine Learning     Validation               Approach   
310  Machine Learning     Validation  Important to Remember   
311  Machine Learning     Validation          Lesson Learnt   
312  Machine Learning     Validation              Algorithm   
313  Machine Learning     Validation               Function   
314  Machine Learning     Validation              Procedure   
315  Machine Learning     Validation              Procedure   
316  Machine Learning     Validation              Procedure   
317  Machine Learning     Validation                    TBD   
5    Machine Learning     Definition       Cross Validation   
315  Machine Learning     Validation              Procedure   
316  Machine Learning     Validation              Procedure   

                                            Definition

### Examples of How Information is Included

In [4]:
d_learning_notes[d_learning_notes['Word']=='Regularization']

,Category,Categorization,Word,Definition,CY_RANK,CZ_RANK,WORD_RANK
144,Machine Learning,Definition,Regularization,Technique used to prevent overfitting by addin...,0.0,0.0,NaN
390,Machine Learning,Model,Regularization,ElasticNet: Approach which combines L1 and L2 ...,0.0,6.0,NaN
391,Machine Learning,Model,Regularization,Lasso: Technique used to prevent overfitting b...,0.0,6.0,NaN
392,Machine Learning,Model,Regularization,Ridge: Technique used to prevent overfitting b...,0.0,6.0,NaN


In [8]:
def d_learning_notes_guide(word_list=[],
                           df=d_learning_notes,
                           notes=notes,
                           definitions=definitions):

    if len(word_list)==0:
        word_list= ['Machine Learning','Model','Bias - Variance Trade Off','Linear Regression']
    
    for word in word_list:
        print('\n############################### New Word ###############################\n')
        print(f'Word: {word}')
        
        for key,data_set in {'notes':notes,'definitions':definitions,'d_learning_notes':d_learning_notes}.items():
            for column in ['Category','Categorization','Word','Definition']:
                try:
                    temp = data_set[data_set[column]==word][['Category','Categorization','Word','Definition']].copy()
                except:
                    temp = data_set[data_set[column]==word].copy()
                
                if len(temp)>0:    
                    print(f'\nRecord Found in Dataset:{key} and Column:{column}')
                    display(temp.head(10))
                else:
                    pass
                
d_learning_notes_guide(['Regularization'])


############################### New Word ###############################

Word: Regularization

Record Found in Dataset:definitions and Column:Categorization


,Category,Categorization,Word,Definition
266,Model,Regularization,ElasticNet,Approach which combines L1 and L2 Regularization.
267,Model,Regularization,Lasso,Technique used to prevent overfitting by addin...
268,Model,Regularization,Ridge,Technique used to prevent overfitting by addin...



Record Found in Dataset:definitions and Column:Word


,Category,Categorization,Word,Definition
233,Machine Learning,Definition,Regularization,Technique used to prevent overfitting by addin...



Record Found in Dataset:d_learning_notes and Column:Word


,Category,Categorization,Word,Definition
144,Machine Learning,Definition,Regularization,Technique used to prevent overfitting by addin...
390,Machine Learning,Model,Regularization,ElasticNet: Approach which combines L1 and L2 ...
391,Machine Learning,Model,Regularization,Lasso: Technique used to prevent overfitting b...
392,Machine Learning,Model,Regularization,Ridge: Technique used to prevent overfitting b...


In [87]:
def d_learning_notes_guide(example_word_dict={},
                           df=d_learning_notes,
                           notes=notes,
                           definitions=definitions):

    if len(example_word_dict)==0:
        example_word_dict= {'Category':'Machine Learning',
                            'Categorization':'Model',
                            'Word':'Bias - Variance Trade Off',
                            'Definition':'Linear Regression'}
    
    for key,word in example_word_dict.items():
        print('\n############################### New Word ###############################\n')
        print(f'Word: {word}')
        print(f'Word Classification: {key}')
    
        for key,data_set in {'notes':notes,'definitions':definitions,'d_learning_notes':d_learning_notes}.items():
    
            print(f"\nInclusion in {key}:")
            for column in ['Category','Categorization','Word','Definition']:
                try:
                    temp = data_set[data_set[column]==word][['Category','Categorization','Word','Definition']].copy()
                except:
                    temp = data_set[data_set[column]==word].copy()
                
                print(f'\nAs Word in {column}')
                
                if len(temp)>0:    
                    display(temp.head(10))
                else:
                    print(f'No Records Found')
                
d_learning_notes_guide()


############################### New Word ###############################

Word: Machine Learning
Word Classification: Category

Inclusion in notes:

As Word in Category


,Category,Categorization,Word,Definition
0,Machine Learning,Problem Definition,Goal,Need this to Pull from Somewhere Else.
1,Machine Learning,Problem Definition,Approach,
2,Machine Learning,Problem Definition,Important to Remeber,
3,Machine Learning,Problem Definition,Lesson Learnt,
4,Machine Learning,Problem Definition,Algorithm,
5,Machine Learning,Problem Definition,Function,
6,Machine Learning,Problem Definition,Procedure,
7,Machine Learning,Problem Definition,TBD,
8,Machine Learning,Data Collection,Goal,Need this to Pull from Somewhere Else.
9,Machine Learning,Data Collection,Approach,



As Word in Categorization
No Records Found

As Word in Word
No Records Found

As Word in Definition
No Records Found

Inclusion in definitions:

As Word in Category


,Category,Categorization,Word,Definition
0,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...
1,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...
2,Machine Learning,Definition,Bias,Bias refers to the systematic error in a model...
3,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...
4,Machine Learning,Definition,Cross Validation,Model evaluation technique used to assess how ...
5,Machine Learning,Definition,Curse of Dimensionality,"As dimensionality grows, data points become in..."
6,Machine Learning,Definition,Deep Learning,Deep learning is a subset of machine learning ...
7,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
8,Machine Learning,Definition,Optimization,Optimization is the process of finding the bes...
9,Machine Learning,Definition,Reinforcement Learning,Reinforcement learning trains an agent to make...



As Word in Categorization
No Records Found

As Word in Word


...


As Word in Definition
No Records Found

Inclusion in d_learning_notes:

As Word in Category
No Records Found

As Word in Categorization
No Records Found

As Word in Word


,Category,Categorization,Word,Definition
4,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...



As Word in Definition
No Records Found

############################### New Word ###############################

Word: Linear Regression
Word Classification: Definition

Inclusion in notes:

As Word in Category
No Records Found

As Word in Categorization
No Records Found

As Word in Word
No Records Found

As Word in Definition
No Records Found

Inclusion in definitions:

As Word in Category
No Records Found

As Word in Categorization
No Records Found

As Word in Word


,Category,Categorization,Word,Definition
109,Model,Algorithm,Linear Regression,Models the relationship between an independent...



As Word in Definition
No Records Found

Inclusion in d_learning_notes:

As Word in Category
No Records Found

As Word in Categorization
No Records Found

As Word in Word
No Records Found

As Word in Definition
No Records Found


In [ ]:
def definition_dq(df,exclude_rule1=['Machine Learning']):
    
    '''
    
    
    '''
    
    df = df.copy()
    
    # Rule 1: Word and Category Can not Equal
    df['Rule1'] = np.where((df['Category']==df['Word'])&(~df['Category'].isin(exclude_rule1)),1,0)
    
    # Rule 2: There should be No Duplication in Word Column
    df['Rule2'] = df.groupby('Word').transform('size')
    df['Rule2'] = np.where(df['Rule2']>1,1,0)

    # Rule 3: All Categories in MUST have a record in Categorization.
    
    
    
    # Identify Rule Violations
    col_list =  [x for x in df.columns if x.find('Rule')!=-1]
    df['Violation'] = df[col_list].sum(axis=1)
    df['Violation'] = np.where(df['Violation']>0,1,0)

    
        
    return df.drop(['Notes','Link','Image','Markdown Equation','Dataset Size','Learning Type','Algorithm Class'],axis=1)

test = definition_dq(definitions)

test[test['Violation']==1]